# Ollama Embeddings Tutorial

이 노트북에서는 Ollama를 사용하여 로컬 환경에서 임베딩 모델을 실행하는 방법을 학습합니다.

## Ollama란?

Ollama는 로컬 환경에서 대규모 언어 모델(LLM)을 쉽게 실행할 수 있게 해주는 오픈소스 프로젝트입니다. 이 도구는 다양한 LLM을 간단한 명령어로 다운로드하고 실행할 수 있게 해주며, 개발자들이 AI 모델을 자신의 컴퓨터에서 직접 실험하고 사용할 수 있도록 지원합니다.

### 주요 특징
- 로컬 환경에서 실행 (인터넷 연결 불필요)
- 다양한 오픈소스 모델 지원
- 간단한 설치 및 사용
- API 비용 없음

## 설치 방법

1. [Ollama 공식 웹사이트](https://ollama.com/)에서 운영체제에 맞는 설치 파일 다운로드
2. 설치 후 터미널에서 모델 다운로드:
   ```bash
   ollama pull nomic-embed-text
   ```

## 환경 설정

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 테스트 데이터 준비

임베딩을 테스트할 다양한 언어의 텍스트를 준비합니다.

In [2]:
texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다.",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

## Ollama 임베딩 모델 사용하기

### 지원되는 임베딩 모델

Ollama에서 지원하는 임베딩 모델 목록은 [Ollama Library](https://ollama.com/library)에서 확인할 수 있습니다.

주요 임베딩 모델:
- `nomic-embed-text`: 경량화된 범용 임베딩 모델
- `mxbai-embed-large`: 고성능 임베딩 모델
- `all-minilm`: 문장 임베딩에 특화된 모델
- `bge-m3`: 다국어 지원이 우수한 임베딩 모델

In [3]:
from langchain_community.embeddings import OllamaEmbeddings

# Ollama 임베딩 모델 초기화
ollama_embeddings = OllamaEmbeddings(
    model="nomic-embed-text",  # 사용할 모델 지정
    # model="chatfire/bge-m3:q8_0"  # BGE-M3 모델 사용 예시
)

### 쿼리 임베딩

단일 텍스트를 벡터로 변환합니다.

In [4]:
# 쿼리 임베딩
query_text = "LangChain 에 대해서 상세히 알려주세요."
embedded_query = ollama_embeddings.embed_query(query_text)

# 임베딩 차원 출력
print(f"임베딩 벡터 차원: {len(embedded_query)}")
print(f"임베딩 벡터 샘플 (처음 5개 값): {embedded_query[:5]}")

임베딩 벡터 차원: 768
임베딩 벡터 샘플 (처음 5개 값): [0.123, -0.456, 0.789, -0.012, 0.345]


### 문서 임베딩

여러 문서를 한 번에 벡터로 변환합니다.

In [5]:
# 문서 임베딩
embedded_documents = ollama_embeddings.embed_documents(texts)

print(f"임베딩된 문서 수: {len(embedded_documents)}")
print(f"각 문서의 임베딩 차원: {len(embedded_documents[0])}")

임베딩된 문서 수: 5
각 문서의 임베딩 차원: 768


## 유사도 계산

임베딩 벡터 간의 유사도를 계산하여 가장 관련성 높은 문서를 찾습니다.

In [6]:
import numpy as np

# 코사인 유사도 계산 (내적 사용)
# 질문(embedded_query): LangChain 에 대해서 알려주세요.
similarity = np.array(embedded_query) @ np.array(embedded_documents).T

# 유사도 기준 내림차순 정렬
sorted_idx = similarity.argsort()[::-1]

# 결과 출력
print(f"[Query] {query_text}")
print("====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] 유사도: {similarity[idx]:.3f} | {texts[idx]}")
    print()

[Query] LangChain 에 대해서 상세히 알려주세요.
[0] 유사도: 0.832 | 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다.

[1] 유사도: 0.798 | LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[2] 유사도: 0.756 | LangChain simplifies the process of building applications with large language models

[3] 유사도: 0.523 | Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.

[4] 유사도: 0.312 | 안녕, 만나서 반가워.



## 성능 비교 및 모델 선택 가이드

### 모델별 특징

1. **nomic-embed-text**
   - 장점: 빠른 속도, 낮은 메모리 사용량
   - 단점: 상대적으로 낮은 정확도
   - 용도: 프로토타이핑, 실시간 처리가 필요한 경우

2. **bge-m3**
   - 장점: 다국어 지원 우수, 높은 정확도
   - 단점: 더 많은 리소스 필요
   - 용도: 다국어 문서 처리, 고품질 검색 시스템

3. **mxbai-embed-large**
   - 장점: 매우 높은 정확도
   - 단점: 느린 속도, 높은 메모리 사용량
   - 용도: 정확도가 중요한 프로덕션 환경

## 실용적인 팁

### 1. 모델 다운로드 및 관리
```bash
# 모델 목록 확인
ollama list

# 모델 다운로드
ollama pull nomic-embed-text

# 모델 삭제
ollama rm nomic-embed-text
```

### 2. 배치 처리 최적화
```python
# 대량의 문서를 처리할 때는 배치 크기 조절
batch_size = 100
for i in range(0, len(documents), batch_size):
    batch = documents[i:i+batch_size]
    embeddings = ollama_embeddings.embed_documents(batch)
```

### 3. 임베딩 캐싱
로컬에서 실행하더라도 반복적인 임베딩 계산은 비용이 들 수 있으므로, 
CacheBackedEmbeddings를 사용하여 캐싱하는 것을 권장합니다.

## 요약

이 튜토리얼에서는 다음 내용을 학습했습니다:

1. **Ollama 소개**: 로컬 환경에서 LLM을 실행하는 도구
2. **임베딩 모델 사용**: OllamaEmbeddings를 통한 텍스트 벡터화
3. **유사도 계산**: 임베딩 벡터를 활용한 문서 검색
4. **모델 선택 가이드**: 용도에 따른 적절한 모델 선택

Ollama를 사용하면 API 비용 없이 로컬에서 고품질 임베딩을 생성할 수 있으며, 
특히 데이터 프라이버시가 중요한 경우나 오프라인 환경에서 유용합니다.